In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
data = pd.read_parquet("/kaggle/input/ecup-comp/train_data.parquet", engine='pyarrow')

In [3]:
df = data

In [4]:
df = df.fillna('unk')

In [5]:
df.sample(3)

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,text_1,text_2,name_1,description_1,name_bert_1,name_2,description_2,name_bert_2
1055430,431438952,431431766,0,"[[0.32259148359298706, -0.3719601035118103, -0...","[[-0.4835028350353241, 0.3550106883049011, 0.2...",EPG Спорт и отдых Аксессуары и принадлежности ...,EPG Спорт и отдых Аксессуары и принадлежности ...,"Блесна-Цикада Strike Pro Farfalla 26, (JG-007C...",Код модели:JG-007C;Название модели:Farfalla 26...,"[-0.17302890121936798, 0.5091333389282227, 0.5...","Блесна-Цикада Strike Pro Farfalla 18, (JG-007A...",Код цвета:790E;Код модели:JG-007A;Название мод...,"[-0.40375131368637085, 0.38520029187202454, 0...."
31626,630709796,1539697576,1,"[[-0.2554186284542084, 0.2284543216228485, 1.0...","[[-0.2554186284542084, 0.2284543216228485, 1.0...",EPG Автотовары Автозапчасти Основные автозапча...,EPG Автотовары Автозапчасти Основные автозапча...,Наконечник рулевой тяги SSANGYONG Istana 95 пр...,PATRON – глобальный бренд запчастей для легков...,"[-0.4938025176525116, 0.5667843222618103, 0.41...","PATRON Наконечник рулевой, арт. PS1403KOR, 1 шт.",PATRON – глобальный бренд запчастей для легков...,"[-0.5091789364814758, 0.5429915189743042, 0.41..."
810732,668982584,1031932281,1,"[[0.35472506284713745, 0.5804797410964966, 0.1...","[[0.35472506284713745, 0.5804797410964966, 0.1...",EPG Автотовары Автозапчасти Автозапчасти подве...,EPG Автотовары Автозапчасти Автозапчасти подве...,"PATRON Стойка стабилизатора, арт. PS4436R",PATRON – глобальный бренд запчастей для легков...,"[-0.5191690325737, 0.5021190047264099, 0.53737...","PATRON Стойка стабилизатора, арт. PS4436R",PATRON – глобальный бренд запчастей для легков...,"[-0.5191690325737, 0.5021190047264099, 0.53737..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168516 entries, 0 to 1168515
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   variantid1        1168516 non-null  int64 
 1   variantid2        1168516 non-null  int64 
 2   target            1168516 non-null  int64 
 3   pic_embeddings_1  1168516 non-null  object
 4   pic_embeddings_2  1168516 non-null  object
 5   text_1            1168516 non-null  object
 6   text_2            1168516 non-null  object
 7   name_1            1168516 non-null  object
 8   description_1     1168516 non-null  object
 9   name_bert_1       1168516 non-null  object
 10  name_2            1168516 non-null  object
 11  description_2     1168516 non-null  object
 12  name_bert_2       1168516 non-null  object
dtypes: int64(3), object(10)
memory usage: 115.9+ MB


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [9]:
vectorizer = TfidfVectorizer(max_features=64)

texts_name_desc_attr = df.apply(lambda x: [x['name_1'] + " " + x['description_1'] +" "+ x['text_1'], x['name_2'] + " " + x['description_2'] + " " + x['text_2']], axis=1).tolist()

tf_feat_name_desc_attr = vectorizer.fit_transform([' '.join(text) for text in texts_name_desc_attr]).toarray()

df['tf_feat_name_desc'] = list(tf_feat_name_desc_attr)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168516 entries, 0 to 1168515
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   variantid1         1168516 non-null  int64 
 1   variantid2         1168516 non-null  int64 
 2   target             1168516 non-null  int64 
 3   pic_embeddings_1   1168516 non-null  object
 4   pic_embeddings_2   1168516 non-null  object
 5   text_1             1168516 non-null  object
 6   text_2             1168516 non-null  object
 7   name_1             1168516 non-null  object
 8   description_1      1168516 non-null  object
 9   name_bert_1        1168516 non-null  object
 10  name_2             1168516 non-null  object
 11  description_2      1168516 non-null  object
 12  name_bert_2        1168516 non-null  object
 13  tf_feat_name_desc  1168516 non-null  object
dtypes: int64(3), object(11)
memory usage: 124.8+ MB


In [11]:
joblib.dump(vectorizer, 'vectorizer_64.pkl')

['vectorizer_64.pkl']

In [11]:
df.sample(5)

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,text_1,text_2,name_1,description_1,name_bert_1,name_2,description_2,name_bert_2,tf_feat_name_desc
778886,1008013515,1008013549,0,"[[0.48579275608062744, 0.14940062165260315, -1...","[[0.27555200457572937, -0.0473518967628479, -0...",Одежда и обувь Обувь Повседневная обувь Кроссо...,Одежда и обувь Обувь Повседневная обувь Кроссо...,Кроссовки THOM BROWNE,unk,"[-0.45168188214302063, 0.5185710787773132, 0.4...",Кроссовки THOM BROWNE,unk,"[-0.45168188214302063, 0.5185710787773132, 0.4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
719851,1422624242,1525666716,1,"[[0.6121782064437866, -0.2620956003665924, -0....","[[0.730206310749054, 0.02066957950592041, -0.6...","Книги и цифровые книги Книги Печатные книги, ж...","Книги и цифровые книги Книги Печатные книги, ж...",Виммельбух с окошками. Детский сад,Виммельбух с окошками «Детский сад» — это боль...,"[-0.3841635584831238, 0.6107991337776184, 0.65...",Виммельбух с окошками. Детский сад,\nВиммельбух с окошками. Детский сад\n<p>Вимме...,"[-0.3841635584831238, 0.6107991337776184, 0.65...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.053..."
251664,1062588818,1062577720,0,"[[-0.5595528483390808, 0.2606722414493561, 0.3...","[[-0.5595528483390808, 0.2606722414493561, 0.3...",Одежда и обувь Одежда Спецодежда Рабочая одежд...,Одежда и обувь Одежда Спецодежда Рабочая одежд...,Спец.одежда летняя Костюм муж. л26-КПК сер/чер...,Спецодежда летняя костюм мужской л26-КПК состо...,"[-0.6078574657440186, 0.5384911894798279, 0.59...",Спец.одежда летняя Костюм муж. л26-КПК сер/чер...,Спецодежда летняя костюм мужской л26-КПК состо...,"[-0.6265439391136169, 0.5318329334259033, 0.57...","[0.0, 0.0, 0.0, 0.0, 0.1988662519327462, 0.0, ..."
553470,1503047040,1503061937,0,"[[0.7529357671737671, -0.030867330729961395, 0...","[[0.7529357671737671, -0.030867330729961395, 0...",EPG Строительство и ремонт Крепеж и фурнитура ...,EPG Строительство и ремонт Крепеж и фурнитура ...,Фасадный дюбель под ключ с фланцем TSX-500 10х...,Используется для крепления различных металличе...,"[-0.30198854207992554, 0.3051392138004303, 0.4...",Фасадный дюбель под ключ с фланцем TSX-500 10х...,Используется для крепления различных металличе...,"[-0.32529595494270325, 0.288170725107193, 0.46...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
558507,769921826,1071266004,1,"[[0.8229413628578186, 0.2710004150867462, 0.69...","[[0.8229413628578186, 0.2710004150867462, 0.69...",Одежда и обувь Одежда Белье Колготки без шорти...,Одежда и обувь Одежда Белье Колготки без шорти...,"Колготки Incanto, 150 ден",Колготки Cotton Club 150 Incanto. Плотные тёпл...,"[-0.332826167345047, 0.6618589758872986, 0.517...","Колготки Incanto, 150 ден",Колготки Cotton Club 150 Incanto. Плотные тёпл...,"[-0.332826167345047, 0.6618589758872986, 0.517...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25269920527279277,..."


In [12]:
embs_pic_cols_1 = [f'pic_emb_1_{i}' for i in range(128)]
embs_pic_cols_2 = [f'pic_emb_2_{i}' for i in range(128)]

new_embs_cols = [f'tfidf_emb_{i}' for i in range(64)]


name_bert_cols_1 = [f'name_bert_1_{i}' for i in range(64)]
name_bert_cols_2 = [f'name_bert_2_{i}' for i in range(64)]

In [13]:
list_emb_pic_1= []
for i in range(df.shape[0]):
    list_emb_pic_1.append(df['pic_embeddings_1'].iloc[i][0])
    
list_emb_pic_2= []
for i in range(df.shape[0]):
    list_emb_pic_2.append(df['pic_embeddings_2'].iloc[i][0])

In [15]:
pic_data_1 = pd.DataFrame(list_emb_pic_1, columns=embs_pic_cols_1)
pic_data_2 = pd.DataFrame(list_emb_pic_2, columns=embs_pic_cols_2)

new_emb_data_cols = pd.DataFrame(df['tf_feat_name_desc'].tolist(), columns=new_embs_cols)
del new_embs_cols
gc.collect()
name_bert_data_1 = pd.DataFrame(df['name_bert_1'].tolist(), columns=name_bert_cols_1)
name_bert_data_2 = pd.DataFrame(df['name_bert_2'].tolist(), columns=name_bert_cols_2)
del name_bert_cols_1
del name_bert_cols_2
gc.collect()


0

In [23]:
df = df[['variantid1', 'variantid2', 'target']]

In [26]:
dataset = pd.concat([df, name_bert_data_1, name_bert_data_2,pic_data_1,pic_data_2, new_emb_data_cols], axis=1)

In [27]:
dataset.head()

,variantid1,variantid2,target,name_bert_1_0,name_bert_1_1,name_bert_1_2,name_bert_1_3,name_bert_1_4,name_bert_1_5,name_bert_1_6,...,tfidf_emb_54,tfidf_emb_55,tfidf_emb_56,tfidf_emb_57,tfidf_emb_58,tfidf_emb_59,tfidf_emb_60,tfidf_emb_61,tfidf_emb_62,tfidf_emb_63
0,1447875869,1447872068,1,-0.392746,0.490946,0.568983,-0.634171,0.630336,0.539259,-0.007866,...,0.000000,0.226706,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,1176231201,284733670,1,-0.540443,0.315268,0.365918,-0.260832,0.429378,0.579384,-0.140100,...,0.122553,0.282688,0.0,0.0,0.086359,0.038451,0.075493,0.0,0.0,0.0
2,658617865,549848659,0,-0.476705,0.549247,0.419049,-0.535690,0.415207,0.591307,-0.296678,...,0.000000,0.000000,0.0,0.0,0.077897,0.000000,0.000000,0.0,0.0,0.0
3,719320625,719370486,1,-0.611602,0.406792,0.549095,-0.485067,0.545327,0.559615,0.171071,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,1067645658,949954740,0,-0.616424,0.676962,0.732529,-0.443149,0.356350,0.590263,0.144103,...,0.306846,0.000000,0.0,0.0,0.432448,0.000000,0.000000,0.0,0.0,0.0


In [29]:
del df
del name_bert_data_1
del name_bert_data_2
del pic_data_1
del pic_data_2
del new_emb_data_cols
del data
del texts_name_desc_attr
gc.collect()

1247

In [33]:
dataset.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168516 entries, 0 to 1168515
Columns: 451 entries, variantid1 to tfidf_emb_63
dtypes: float64(448), int64(3)
memory usage: 3.9 GB


In [43]:
dataset.to_parquet('dataset.parquet', engine='pyarrow')

In [44]:
dataset.shape

(1168516, 451)